In [2]:
import traceback
import sys,glob
import os.path

import numpy as np
from datetime import date
import netCDF4
import time
from numba import njit
sys.path.append('../')
from utils import *
from anomaly import *
import matplotlib.pylab as plt
import scipy.stats
import f90nml
import xarray as xr
sys.path.append('../../cds-backend/code/')
import cds_eua3 as eua
import urllib3
import json
import h5py
import copy
from functools import partial
from multiprocessing import Pool

In [3]:
plist=[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500,100000]

In [4]:
data=dict(zip(['d','u','v'],eua.vm_request_wrapper({'variable': ['wind_direction','u_component_of_wind','v_component_of_wind'],  
                                                                'optional':['obs_minus_bg','bias_estimate'],
                                                                'statid': '11035', 
                                                                'pressure_level':plist,
                                                                }, 
                                        overwrite=True,vm_url='http://srvx8.img.univie.ac.at:8002').values()))


VM Request failed {'variable': ['wind_direction', 'u_component_of_wind', 'v_component_of_wind'], 'optional': ['obs_minus_bg', 'bias_estimate'], 'statid': '11035', 'pressure_level': [1000, 2000, 3000, 5000, 7000, 10000, 15000, 20000, 25000, 30000, 40000, 50000, 70000, 85000, 92500, 100000]}


ConnectionError: HTTPConnectionPool(host='srvx8.img.univie.ac.at', port=8002): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5ea0b9cf10>: Failed to establish a new connection: [Errno 111] Connection refused'))